## ВКР

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

### Библиотеки

selenium поскольку нужен js

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit

### Скрейпинг страницы со списком резюме

hh.ru не принимает названия с заглавными буквами:

In [64]:
def normalize_translit(ru_query):
    text = translit(ru_query, language_code='ru', reversed=True)
    text = text.replace(' ', '-').replace('yj', 'yy').replace('yy-', 'yy_').replace('es-', 'es_').replace('k-pr', 'k_pr')  # эвристическое правило, не знаю, почему у hh так...
    return text

In [ ]:
driver = webdriver.Chrome('./chromedriver')

In [65]:
def scrape(query, driver):
    # ищем кол-во страниц
    pages_count = 10  # соскрейпить это число иначе ошибки!
    link = 'https://hh.ru/resumes/{}?items_on_page=100&page={}'.format(query, 0)
    driver.get(link)
    
    pages_cnt_site = driver.find_elements(by=By.XPATH, value='//span[@class="pager-item-not-in-short-range"]/a[@class="bloko-button"][@rel="nofollow"][@data-qa="pager-page"]/span')
    if pages_cnt_site:
        pages_count = int(pages_cnt_site[-1].text)
        # print(pages_count)
    
    for n in range(pages_count):  # проблема!!
        link = 'https://hh.ru/resumes/{}?items_on_page=100&page={}'.format(query, n)  # 100 per page
        driver.get(link)

        resumes_list = driver.find_element(by=By.XPATH, value='//div[@class="resume-search-item__header"]')
        # resumes_list.click()  # стоп а зачем это...
        resumes = resumes_list.find_element(by=By.XPATH, value='//div[@data-qa="resume-serp__results-search"]').find_elements(by=By.XPATH, value='//a[@class="serp-item__title"]')

        resumes_parsed = []
        for x in resumes:
            resumes_parsed.append((x.text, x.get_attribute("href")))

        if n == 0:
            df = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])
            # print(len(resumes_parsed))
        else:
            # print(len(resumes_parsed))
            df2 = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])  # , index=list(range(0 + 100 * n, 100 + 100 * n)) n00, ..., n99n = 0, ..., 49
            df = pd.concat([df, df2])
    
    return df

<ipython-input-65-3541302dc275>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [66]:
analysts_list = ['аналитик bi',
                'системный аналитик',
                'бизнес аналитик',
                'аналитик продаж',
                'финансовый аналитик',
                'аналитик данных',
                'data analyst']
analysts_map = dict()

for analyst in analysts_list:
    analysts_map[analyst] = scrape(normalize_translit(analyst), driver)
    analysts_map[analyst].index = range(analysts_map[analyst].shape[0])

Конкатеннация результатов:

In [83]:
df1 = pd.DataFrame()

for analyst in analysts_list:
    df2 = analysts_map[analyst]
    df1 = pd.concat([df1, df2])
    
df1.index = range(df1.shape[0])
df1.to_csv('resumes_all.csv')

### Скрейпинг страницы с конкретным резюме

In [68]:
def normalize(x):
    if x:
        x = x[0].text
    else:
        x = '---'  # так будет лучше чем просто ' '?
    return x

In [78]:
def scrape(link, driver):
    driver.get(link)
    comandirovka = driver.find_elements(by=By.XPATH, value='//div[@class="bloko-translate-guard"]')
    comandirovka = normalize(comandirovka)  # норм?
    
    opyt = driver.find_elements(by=By.XPATH, value='//span[@class="resume-block__title-text resume-block__title-text_sub"]')
    opyt = normalize(opyt)

    about = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-container"][@data-qa="resume-block-skills-content"]')
    about = normalize(about)
        
    educ = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block"][@data-qa="resume-block-education"]')
    educ = normalize(educ)
        
    inter = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-item-gap"]')
    inter = normalize(inter)
        
    inter2 = driver.find_elements(by=By.XPATH, value='//div[@class="key-skills-row"][@data-qa="tags-key-skills"]')
    inter2 = normalize(inter2)
    
    # Образование; все написано не в том же стиле что и ^
    
    educ_val = '//div[@data-qa="resume-block-education"][@class="resume-block"]//div[@data-qa="resume-block-education-name"]'
    educ_ = driver.find_elements(by=By.XPATH, value=educ_val)
    stepen_val = '//div[@data-qa="resume-block-education"]//span[@class="resume-block__title-text resume-block__title-text_sub"]'
    stepen = driver.find_elements(by=By.XPATH, value=stepen_val)
    title_val = '//span[@class="resume-block__title-text"][@data-qa="resume-block-title-position"]'
    title = driver.find_elements(by=By.XPATH, value=title_val)
    if title:
        title = title[0].text
    else:
        title = ''
    
    educ1 = ''
    educ2 = ''
    educ3 = ''
    
    if educ_:
        educ1 = educ_[0].text
    if len(educ_) > 1:
        educ2 = educ_[1].text
    if stepen:
        educ3 = stepen[0].text
    vuzes_count = len(educ_)
    
    # Проблема: иногда уровень образования могут не писать
    
    # Работы
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    work1 = '---'
    work2 = '---'
    
    if len(work) > 0:
        work1 = work[0].text
    if len(work) > 1:
        work2 = work[1].text
    
    # EXPERIMENTAL: пробую собрать work
    work_list = []
    for w in work:
        work_list.append(w.text)
    work_count = len(work)
        
    return (link, title, comandirovka, opyt, about, educ, inter, inter2, educ1, educ2, 
            educ3, vuzes_count, work1, work2, work_list, work_count)

In [79]:
# driver = webdriver.Chrome('./chromedriver')

In [84]:
df_links = list(pd.read_csv('resumes_all.csv')['Ссылка'])

resume_list = []
for link in df_links:
    # print(link)
    data = scrape(link, driver)
    resume_list.append(data)

df = pd.DataFrame(resume_list, columns=['Ссылка',
                                        'Название',
                                        'Коммандировка', 
                                        'Опыт работы, лет',
                                        'О себе',
                                        'Образование',
                                        'Интересы',
                                        'Навыки',
                                        'Образование-1',
                                        'Образование-2',
                                        'Уровень образование',
                                        'Кол-во образований',
                                        'Работа 1',
                                        'Работа 2',
                                        'Где работал?',
                                        'Кол-во работ'])
df.to_csv('resumes_features.csv')
df.head(2)

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,Уровень образование,Кол-во образований,Работа 1,Работа 2
0,https://hh.ru/resume/1aa91e410000fd3cda0039ed1...,Analyst,"Moscow, willing to relocate, prepared for occa...",Work experience 7 years 4 months,"Responsible, communicable, quick study and det...",Higher education\n2015\nMOSCOW STATE UNIVERSIT...,"Specializations:\nSales manager, account manag...",Поддержка продаж / владение ПО для поддержки п...,MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEER...,,Higher education,1,"NUTRICIA\nMoscow, nutricia.ru\nFood Products.....","Volkswagen Group Russia\nMoscow, www.volkswage..."
1,https://hh.ru/resume/628596ac000657935b0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к редким ...",Опыт работы 1 год 5 месяцев,В последние годы проходила обучение без возмож...,Высшее образование (Бакалавр)\n2022\nНациональ...,"Специализации:\nBI-аналитик, аналитик данных\n...","Работа с компьютерными программами, программы ...","Национальный исследовательский университет ""Вы...",,Высшее образование (Бакалавр),1,"Ozon\nМладший аналитик\nСоздание, поддержка и ...",OZON\nСтажер группы BI аналитики и отчетности\...


## Черновик

**Справочник по образованию:**

Найти подобные списки для ТОП-10 вузов и проанализировать:
https://propostuplenie.ru/article/abbreviatury-i-prozvishcha-populyarnyh-vuzov/?ysclid=la9p7r8feo708199013

Топ вузов: https://vuzoteka.ru/вузы?ysclid=la9p8mfldl600081011

**Конкатеннация таблиц:**

Что если использовать тупой метод:

`df` — таблица, собранная раньше

`df2` — таблица, собранная повторным запусом скрейпера

`new_df = pd.concat([df, df2]).drop_duplicates` ??

попробую так для `df==df2==df_cur`, чтобы посмотреть время работы

**Перевод опыта работы в числовое значение:**

In [5]:
df = pd.read_csv('resumes_features.csv')

In [22]:
df['Опыт'][2]

'Опыт работы 3 года 3 месяца'

In [33]:
for x in df['Опыт']:
    x=x.removeprefix('Work experience ').removeprefix('Опыт работы ')
    x=x.replace('years', ';')
    x=x.replace('year', ';')
    x=x.replace('года', ';')  # ВСЕГДА д. б. раньше
    x=x.replace('год', ';')
    x=x.replace('лет', ';')
    
    x=x.removesuffix('months')
    x=x.removesuffix('month')
    x=x.removesuffix('месяцев')
    x=x.removesuffix('месяц')
    x=x.removesuffix('месяца')
     
    x=x.replace(' ', '')
    print(x)

7;4
1;5
3;3
7;2
1;1
10;7
2;7
2;10
4;1
7;10
7;
6;5
17;
19;3
2;6
7;
6;5
3;4
5;7
12;11
13;10
2;
13;9
2;11
8;3
8
2
2;2
4;3
13;
3;7
2;5
4;2
3;
5;
34;4
10;7
3;3
1;
3;2
3
1;4
7;3
3;4
9;3
4;6
7;3
5;3
1;8
5;4
18;2
10;
4;2
10
2;11
2;4
2;
11;
3;2
1;6
10;5
6;5
1;
4;2
8;8
4;5
4;3
1;8
9;3
1;6
20;5
3;10
14;4
7;4
13;
5;2
3;11
1;5
2;10
10;7
7;2
5;9
5;3
26;6
17;1
10;3
7;
1;4
8;3
13;7
2;1
11;10
1;1
1;3
3;11
14;3
3;7
3;8
1;
8;3
12;9
2;1
1;5
7;1
15;5
2;8
11;7
1;3
26;1
1;7
2;4
14;9
19;2
3;11
4;1
12;2
7
5;5
2;2
22;8
3;
9;8
12;5
7;2
2;11
27;9
23;8
7;9
1;4
5;8
10;
3;2
6;
2;10
4;7
4;3
6;2
4;1
2;3
10;4
16;
9;3
7;1
11;9
6;2
Анкетаменеджерапомаркетингуилирекламе
23;8
6;1
6;
2;11
10;7
1;3
1;8
8;
7;10
17;2
16;5
13;10
5;4
27;9
6;3
2;5
3;11
24;11
24;3
5;6
10;1
2;2
11;11
26;4
17;11
4
16;5
1;10
12;8
17;3
1;5
24;4
Обомне
8;7
14;7
6;9
2;
5;6
3;6
Компетенциииключевыенавыки
7;3
8;1
13;7
4;1
3;4
8;9
3;10
5;5
5;
11
16;10
6;
14;4
1;3
6;6
2;4
5;
8
15;2
3;9
6;6
3;3
16;11
1;10
5;4
12;6
1;2
2;5
1;4
1;10
Компетенциииключевыенавыки


^ остальные значения считаем пропуском

**Словарь образования:**

Сделали сам список:

In [60]:
edu_list = pd.concat([df['Образование-1'], df['Образование-2']], axis=0).drop_duplicates().reset_index().drop('index', axis=1)
edu_list = list(edu_list[list(edu_list)[0]])
len(edu_list)

10153

Как сократить его численность? Посмотрим, в каких городах есть топ вузы и уберем те где эти города не встречаются. англ+рус. отдельно потом рассмотреть то где больше 1 слова — значит это не имя города или больше 1 заглавной

Топ-10 IT-вузов России по версии QS (Quacquarelli Symonds)

Московский государственный университет им. М. В. Ломоносова;

Университет ИТМО;

Московский физико-технологический институт (МФТИ);

Высшая школа экономики;

Санкт-Петербургский политехнический университет Петра Великого; 

Московский государственный технический университета (МГТУ) имени Н.Э. Баумана;

Санкт-Петербургский политехнический университет Петра Великого; 

Новосибирский государственный университет;

Московский инженерно-физический институт (МИФИ);

Томский политехнический университет.

In [61]:
ys=[]

for x in edu_list:
    s=str(x)
    l = s.find(',')+1
    y=s[l:].lstrip(' ')
    print(end='')
    ys.append(y)

In [62]:
for i in range(len(ys)):
    y = ys[i]
    is_space = re.fullmatch('[А-я]+[ ][А-я ()-]+', y)
    if is_space:
        #print(y)
        1==1
    is_m = re.fullmatch('Mosc.*', y) or re.fullmatch('Моск.*', y) or re.fullmatch('Петерб.*', y) or re.fullmatch('Peters.*', y) or re.fullmatch('СПб.*', y) or re.fullmatch('Новосиб.*', y) or re.fullmatch('Novosib.*', y) or re.fullmatch('Томск.*', y) or re.fullmatch('Tomsk.*', y)
    if not is_space and not is_m:
        ys[i] = 'NO'
#ys

In [63]:
cnt=0
for y in ys:
    if y != 'NO':
        cnt+=1
cnt

4825

Итак, все равно куча всего... Теперь избавимся от дубликатов — это же список и необязатлеьно знать номер строки с этим вузом в таблице.

In [64]:
import numpy as np
ys = np.array(ys)
ys=np.unique(ys)
ys=list(ys)
len(ys)

4149

Топ-10 IT-вузов России по версии QS (Quacquarelli Symonds)

1. Московский государственный университет им. М. В. Ломоносова;
2. Университет ИТМО;
3. Московский физико-технологический институт (МФТИ);
4. Высшая школа экономики;
5. Санкт-Петербургский политехнический университет Петра Великого; 
6. Московский государственный технический университета (МГТУ) имени Н.Э. Баумана;
7. Санкт-Петербургский политехнический университет Петра Великого; 
8. Новосибирский государственный университет;
9. Московский инженерно-физический институт (МИФИ);
10. Томский политехнический университет.

https://propostuplenie.ru/article/spisok-luchshih-informacionnyh-vuzov-rossii/

In [71]:
for y in ys:
    if re.search('ВШЭ', y) or re.search('Высшая школа экономики', y) or re.search('HSE', y):
        y='ВШЭ'
    if re.search('Московский Государственный Университет$', y) or re.search('Московский Государственный Университет им[.  ени] (М.|Л)', y) or re.search('МГУ[^ А-я]|\(МГУ\)', y) or re.search('Moscow state University', y) or re.search('MSU', y):
        y='МГУ'
    if re.search('ИТМО', y):
        print(y)
    if re.search('МФТИ', y) or re.search('Московский физико-технологический институт', y):
        print(y)
        # ЗФТШ при МФТИ
        # - Физтех
    if re.search('СПБ Политех', y) or re.search('Санкт-Петербургский политехнический университет', y):
        print(y)
    if re.search('Бауман', y):
        y='Бауманка'
    if re.search('НГУ', y) or re.search('Новосибирский государственный университет', y):
        print(y)
        # НГУ Наталии Нестеровой
    if re.search('МИФИ', y) or re.search('Московский инженерно-физический институт', y):
        print(y)
        # Какие-то Техникум ДИТИ НИЯУ МИФИ и прочее...
    if re.search('ТомГУ', y) or re.search('Томский политехнический университет', y):
        print(y)
    else:
        y='NO'
        
# Дальше завожу колонку: parsed_uni по этому правилу -- 1 если хотя бы 1 уник топовый
# И там заменяю значения и вывожу равно ли МГУ / етц.

ВКИ НГУ
Военная кафедра НИЯУ МИФИ
Высший коледж Информатики при НГУ
Высший колледж информатики НГУ
1 ЗФТШ при МФТИ
МИФИ (ГУ)
МИФИ (Московский Инженерно Физический Институт
МИФИ (Московский Инженерно-Физический Институт)
МИФИ (Московский государственный инженерно-физический университет)
МИФИ (Снежинское отделение)
МИФИ (ТУ)
МИФИ (ЭАИ)
МИФИ - Аспирантура
МИФИ - Московский Инженерно-Физический Институт
МИФИ - Московский Инженерно-физический институт (государственный университет)
МИФИ - Московский инженерно физический институт
МИФИ -- Московский инженерно-физический институт
МИФИ аспирантура
МОСКОВСКИЙ ИНЖЕНЕРНО-ФИЗИЧЕСКИЙ ИНСТИТУТ (ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ) Экономико-аналитический институт МИФИ
1 МФТИ (ГУ)
1 МФТИ (Физтех)
1 МФТИ - Московский Физико-Технический Институт
Математический Колледж при НИЯУ МИФИ
Международная бизнес школа при НГУЭиУ
Московский Государственный Инженерно Физический Институт (МИФИ). Государственный Университет.
Московский Государственный Инженерно-Физический Инс

Он конечно неидеальный но хоть что-то... Это баовый справочник. Потом вручную через cmd+F посмотрб по типу "Политех..." итд посмотрб какие еще варианты написания есть и добавлю сюда

или через re.find('Политех', df)

дальше пишу функцию 

```
def is_top(text):
    if одно из условий выше на text==y:
        return 1
```

**Флаг, что текст на eng:**

In [368]:
df['О себе']
cnt=0

for y in df['О себе']:
    if type(y)!=float:  # not nan
        if re.search('[a–Z]+', y):  # более точно: у английских резюме интересы тоже будут на англ итд, потом подумаю над этим
            cnt+=1
cnt, df.shape[0]-cnt

(6741, 13077)

**Справочник по интересам:**

In [72]:
# ... А что здесб делать???

**Извлечение сущностей:**

In [58]:
"""
1. Коммандировка 
2. Навыки
3. Кол-во работ
4. Наличие аналитических комбинаций слов в работе — новый столбец
"""

import numpy as np
df['Коммандировка'][8]

# (не) готов -> (не) готова
# (not) willing, (not) preprared

# City, (not willing)., (not) prepared

import re

print(df['Коммандировка'][8])
print(re.search('[A-z]+', df['Коммандировка'][8]))
print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][8]))
print(re.search('[a-z ]*prepared for business trips', df['Коммандировка'][8]))

print(df['Коммандировка'][0])
print(re.search('[A-z]+', df['Коммандировка'][0]))
print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][0]))
print(re.search('[a-z ]*prepared for [a-z]* business trips', df['Коммандировка'][0]))
# ^ оказывается есть еще occasional
# тогда для eng-коммандировки готово!

# Коммандировка более-менее есть. Теперь навыки. 

df['Навыки'][0].split('\n')  # вот так?

# Навыки есть. Теперь кол-во работ. Для этого соскрейпить все работы и их кол-во

df

# У меня уже скрейпится список всех работ. Надо его добавить просто а не только 12!!

# Все вроде я это собрал

# Теперь наличие аналитических комбинаций слов в работе. Для этого сначала сделать 3 пункт!!

Moscow, not willing to relocate, prepared for business trips
<re.Match object; span=(0, 6), match='Moscow'>
<re.Match object; span=(7, 31), match=' not willing to relocate'>
<re.Match object; span=(32, 60), match=' prepared for business trips'>
Moscow, willing to relocate, prepared for occasional business trips
<re.Match object; span=(0, 6), match='Moscow'>
<re.Match object; span=(7, 27), match=' willing to relocate'>
<re.Match object; span=(28, 67), match=' prepared for occasional business trips'>


,Unnamed: 0,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,Уровень образование,Кол-во образований,Работа 1,Работа 2
0,0,https://hh.ru/resume/1aa91e410000fd3cda0039ed1...,Analyst,"Moscow, willing to relocate, prepared for occa...",Work experience 7 years 4 months,"Responsible, communicable, quick study and det...",Higher education\n2015\nMOSCOW STATE UNIVERSIT...,"Specializations:\nSales manager, account manag...",Поддержка продаж / владение ПО для поддержки п...,MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEER...,NaN,Higher education,1,"NUTRICIA\nMoscow, nutricia.ru\nFood Products.....","Volkswagen Group Russia\nMoscow, www.volkswage..."
1,1,https://hh.ru/resume/628596ac000657935b0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к редким ...",Опыт работы 1 год 5 месяцев,В последние годы проходила обучение без возмож...,Высшее образование (Бакалавр)\n2022\nНациональ...,"Специализации:\nBI-аналитик, аналитик данных\n...","Работа с компьютерными программами, программы ...","Национальный исследовательский университет ""Вы...",NaN,Высшее образование (Бакалавр),1,"Ozon\nМладший аналитик\nСоздание, поддержка и ...",OZON\nСтажер группы BI аналитики и отчетности\...
2,2,https://hh.ru/resume/dfbcae000004342d8f0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к команди...",Опыт работы 3 года 3 месяца,---,Высшее образование (Бакалавр)\n2019\nСамарский...,"Специализации:\nАналитик\nBI-аналитик, аналити...",Другое\nPython\nSQL\nУмение работать в коллект...,Самарский государственный аэрокосмический унив...,NaN,Высшее образование (Бакалавр),1,"Банк ВТБ (ПАО)\nМосква, www.vtbcareer.com\nФин...",---
3,3,https://hh.ru/resume/bebf6a630008069d8c0039ed1...,Аналитик,"Санкт-Петербург, м. Гражданский проспект, гото...",Опыт работы 7 лет 2 месяца,Имею экономическое образование. Продвинутый по...,Высшее образование (Бакалавр)\n2015\nСанкт-Пет...,Специализации:\nАналитик\nЗанятость: полная за...,"Работа с компьютерными программами, программы ...",Санкт-Петербургский государственный политехнич...,NaN,Высшее образование (Бакалавр),1,A&D Mortgage\nСтарший BI аналитик\nСоздание Po...,"Сбер\nСанкт-Петербург, rabota.sber.ru/\nФинанс..."
4,4,https://hh.ru/resume/8606f37b000773de980039ed1...,Программист-разработчик,"Москва, м. Петровско-Разумовская, не готов к п...",Опыт работы 1 год 1 месяц,Имеется опыт в создании Android-приложений на ...,Высшее образование (Бакалавр)\n2022\nЧелябинск...,"Специализации:\nПрограммист, разработчик\nЗаня...",Применение стека технологий/Цифровые навыки\nL...,"Челябинский государственный университет, Челяб...",NaN,Высшее образование (Бакалавр),1,MoneyCare\nBI-аналитик\nРабота с big data(Pyth...,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19813,19813,https://hh.ru/resume/57e8fd3c000211f20a0039ed1...,Финансовый директор,"Москва, м. Тушинская, не готов к переезду, гот...",Опыт работы 25 лет 3 месяца,"Высокая работоспособность, вдумчивость, уравно...",Высшее образование\n2013\nЕдиная Аттестационна...,Специализации:\nФинансовый менеджер\nЗанятость...,---,Единая Аттестационная Комиссия (ЕАК),Ярославское ВВФУ,Высшее образование,2,"ЗАО ""Аудит БОС""\nМосква, audit-bos.ru\nФинансо...",---
19814,19814,https://hh.ru/resume/bd1a048a0002633bc20039ed1...,Аналитик,"Оренбург, готов к переезду (Москва), не готов ...",Опыт работы 10 лет 8 месяцев,"Увлекаюсь спортом, вредные привычки отсутствую...","Высшее образование\n2011\nОГАУ\nЭкономический,...",Специализации:\nАналитик\nЗанятость: полная за...,Другое\nАналитический склад ума\nОтветственнос...,ОГАУ,NaN,Высшее образование,1,"Сбербанк России\nМосква, www.sbrf.ru\nФинансов...",---
19815,19815,https://hh.ru/resume/516fa73d0002e26ab40039ed1...,"Главный бухгалтер, финансовый директор","Москва, м. Котельники, не готова к переезду, н...",Опыт работы 32 года 9 месяцев,"Уверенный пользователь ПК, работа с оргтехнико...",Высшее образование\n2009\nМосковский финансово...,Специализации:\nБухгалтер\nЗанятость: полная

**Перевод уровня образования:**

In [392]:
z = list(np.unique(list(df['Уровень образование'])))

for y in z:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    print(y)

Высшее образование
Доктор наук
Доктор наук
Бакалавр
Магистр
Неоконченное высшее образование
Среднее образование
Среднее специальное образование
nan
Высшее образование
Бакалавр
Доктор наук
Кандидат наук
Магистр
Неоконченное высшее образование
Образование
Среднее образование
Среднее специальное образование


**Звонок на прошлой неделе:**

1. Перевод опыта работы в числовое значение. год года лет -> years; year, years -> years -> ?
2. Беру словарь названий вузов. 

Дальше доделаю

**Что нужно сделать по звонку:**

- Обсудить справочник образования
- Начать анализ данных
- Оформить документы и LMS

Все остальное есть. 

1. Командировка. Надо продумать — там много данных мб разделить, понадобишься
2. Опыт работы перевести в числовое значение (Years -> годы -> 16)
3. Образованеи: год окончания а отдельный столбец. Беру словарь distinct названий вузов из 20к. МИЭМ, МИЭМ НИУ ВШЭ, Московский институт етц Тихонова … Беру топ-10 и только их добавляю в справочник. Справочник — только для 10 вузов. Не только московскими вузами ограничиваться: СПбГУ итд. Топорность с точки зрения IT. 
4. Интересы: составить справочник тоже (позже)
5. Навыки: отдельно с текстом поработает 
6. Уровень образования: тоже перевести (через справочник)
7. Текстовые модели на RU и ENG разные => мб флаг по тому что текст на английском
8. Кол-во работ!!
9. Дальше поля: были ли в работе аналитические комбинации слов
10. Тогда чищу данные, составляю словари итд

Сделать так:
Raw образование -> все извлеченные из raw text сущности, все приведено к единому виду итд
И так с каждым полем

Random/NLP АДИС — там есть объяснение MT и BERT, мб мне это поможет. 

И!!!! Сейчас его для начала главное научить собирать таблицу со ссылками без удаления данных. А дальше разберусь с фичами.